In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from cell_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk


In [3]:
dataset_settings

,network_model,stain,margins,num_sample_per_class,stacks,cell_features_used
dataset_id,,,,,,
20,Inception-BN,nissl,200/500,1000,MD585,NaN
21,Inception-BN,nissl,200/500,1000,MD589,NaN
22,Inception-BN,nissl,200/500,1000,MD594,NaN
23,inception-bn-blue,nissl,500,1000,MD585,NaN
24,inception-bn-blue,nissl,500,1000,MD589,NaN
25,inception-bn-blue,nissl,500,1000,MD594,NaN
50,inception-bn-blue,neurotrace_blue,500,1000,MD642,NaN
90,NaN,nissl,500,1000,MD589,allSizeHist/huMomentsHist
92,NaN,nissl,500,1000,MD589,largeLargeLinkLenHist


In [4]:
dataset_id = 94

# Get information from the dataset[id]
dataset_properties = dataset_settings.loc[dataset_id]
num_samples_per_label = dataset_properties['num_sample_per_class']
stacks = dataset_properties['stacks'].split('/')
cell_features_used = dataset_properties['cell_features_used'].split('/')

In [5]:
# Generate labels in the training set
structures_to_sample = all_known_structures # Pick the labeled structure name for training

# negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]
margins_to_sample = map(int, str(dataset_properties['margins']).split('/'))

In [11]:
# Identify region indeices by section by label
def identify_predefined_regions_one_section_by_label(stack, sec, labels_to_sample, labeled_contours):
    """
    Returns:
        dict {label: list of region indices}
    """
    
    if is_invalid(stack=stack, sec=sec):
        sys.stderr.write('Regions on section %d are not identified because the section is invalid.\n' % sec)
        return
    
    region_contours = load_cell_classifier_data(what='region_contours', stack=stack, sec=sec, ext='bp')
    
    surround_margins = set([get_margin_from_surround_label(label) for label in labels_to_sample if is_surround_label(label)])
    region_labels = label_regions(stack=stack, section=sec, 
                                  region_contours=region_contours,
                                  surround_margins=surround_margins,
                                  labeled_contours=labeled_contours)
    region_labels = {label: region_indices for label, region_indices in region_labels.iteritems()
                     if len(region_indices) > 0}
    return region_labels

In [12]:
def sample_predefined_regions_one_section_by_label(stack, sec, labels_to_sample, labeled_contours, num_samples_per_label=None):
    """
    Sample region addresses in a particular section and associate them with different labels (positive, surround, negative, noclass, foreground, background, etc.).
    
    Args:
        region_contours (list of nx2 arrays): list of contour vertices.
        labeled_contours (dict of nx2 arrays): {label: contour vertices}
        
    Returns:
        dict of 3-tuple list: {label: list of (stack, section, region_index)}.
        If section is invalid, return None.
    """

    addresses = {}
    
    if is_invalid(stack=stack, sec=sec):
        sys.stderr.write('Regions on section %d are not sampled because the section is invalid.\n' % sec)
        return
    
    try:
        region_labels = load_hdf_v2(DataManager.get_region_labels_filepath(stack=stack, sec=sec))
        region_labels = {label: region_indices for label, region_indices in region_labels.iteritems()
                        if label in labels_to_sample}
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Cannot load pre-computed region labels for section %d... re-compute\n' % sec)
        region_labels = identify_predefined_regions_one_section_by_label(stack, sec, labels_to_sample, labeled_contours)

    for label, region_indices in region_labels.iteritems():
        if num_samples_per_label is None:
            addresses[label] = [(stack, sec, ridx) for ridx in region_indices]
        else:
            sampled_region_indices = np.random.choice(region_indices, min(num_samples_per_label, len(region_indices)), replace=False)
            addresses[label] = [(stack, sec, ridx) for ridx in sampled_region_indices]

    return addresses


In [13]:
def load_cell_based_features_one_section_(stack, section, region_indices, cell_features_used):
    """
    Load pre-computed cell-based features for a list of regions on a particular section.
    """
    
    region_features_all_regions = load_cell_classifier_data(what='region_features', stack=stack, sec=section, ext='hdf')
    # Loading hdf ~ 2 seconds.
    
    all_features_normalized = []
    for feature_name in cell_features_used:
        feats = np.asarray([region_features_all_regions[region_ind][feature_name] for region_ind in region_indices])
        
        # huMomentsHist feats: n_regions x 7 x n_bins
        # other feats: n_regions x n_bins
        
        feats_normalized = feats/feats.sum(axis=-1)[...,None].astype(np.float)
        if feature_name == 'huMomentsHist':
            feats_normalized = feats_normalized.reshape((feats_normalized.shape[0], -1))
        all_features_normalized.append(feats_normalized)

    features = np.hstack(all_features_normalized)
    
    return features


In [14]:
def generate_dataset_cell_based(num_samples_per_label, stacks, labels_to_sample, cell_features_used):
    """
    Generate dataset.
    - Extract addresses
    - Map addresses to features
    - Remove None features
    
    Returns:
        features, addresseslab
    """
    
    # Sample addresses
    
    addresses = defaultdict(list)
    addresses_by_section_by_label = []
    
#     t = time.time()
    
    for stack in stacks:
        
        first_sec, last_sec = metadata_cache['section_limits'][stack]
    
        t1 = time.time()

        contours_df, _ = DataManager.load_annotation_v3(stack=stack)
        labeled_contours = contours_df[(contours_df['orientation'] == 'sagittal') & (contours_df['downsample'] == 1)].drop_duplicates(subset=['section', 'name', 'side', 'filename', 'downsample', 'creator'])
        labeled_contours = convert_annotation_v3_original_to_aligned_cropped(labeled_contours, stack=stack)

        sys.stderr.write('Load annotation. Time: %.2f seconds.\n' % (time.time() - t1))

        t1 = time.time()
        
        # Sample addresses from each section

        # If region labels are already computed, sequential loading is much faster (3s) than parallel loading (50s) due to disk read contention.
        addresses_by_section_by_label_curr_stack = [
            sample_predefined_regions_one_section_by_label(stack=stack, sec=sec, 
                                                                 labels_to_sample=labels_to_sample,
                                                                 labeled_contours=labeled_contours[labeled_contours['section']==sec],
                                                                 num_samples_per_label=30)
            for sec in range(first_sec, last_sec+1)]

#         pool = Pool(NUM_CORES/2)
#         addresses_by_section_by_label_curr_stack = \
#         pool.map(lambda sec: sample_predefined_regions_one_section_by_label(stack=stack, sec=sec, 
#                                                                  labels_to_sample=labels_to_sample,
#                                                                  labeled_contours=labeled_contours[labeled_contours['section']==sec],
#                                                                  num_samples_per_label=30),
#                  range(first_sec, last_sec+1))
#         pool.close()
#         pool.join()
                
        addresses_by_section_by_label += addresses_by_section_by_label_curr_stack

        sys.stderr.write('Sample addresses (stack %s): %.2s seconds.\n' % (stack, time.time() - t1))
        
    # Aggregate addresses sampled form each section
    
    t = time.time()
    
    addresses_by_label = defaultdict(list)
    for addrs_by_label in addresses_by_section_by_label:
        if addrs_by_label is not None: # handle cases of invalid sections for which sampling function returns None
            for label, addrs in addrs_by_label.iteritems():
                addresses_by_label[label] += addrs
    addresses_by_label.default_factory = None
    
    if num_samples_per_label is not None:
        import random
        addresses_by_label = {label: random.sample(addrs, min(num_samples_per_label/len(stacks), len(addrs))) 
                              for label, addrs in addresses_by_label.iteritems()}

    # Remove unwanted labels
    addresses_by_label = {label: addrs for label, addrs in addresses_by_label.iteritems() if label in labels_to_sample}
    
    sys.stderr.write('Aggregate addresses: %.2f seconds\n' % (time.time() - t))    
    
    # Map addresses to features
    
    t = time.time()
    
    load_cell_based_features_given_address_list = lambda addrs: smart_map(addrs, keyfunc=lambda (st, se, ri): (st, se),\
                        func=lambda (st, se), gr: \
                        load_cell_based_features_one_section_(stack=st, section=se, 
                                                              region_indices=[ri for _,_,ri in gr], cell_features_used=cell_features_used))
    features_by_label = apply_function_to_dict(load_cell_based_features_given_address_list, addresses_by_label)
    features_by_label = apply_function_to_dict(np.asarray, features_by_label)
    
    sys.stderr.write('Map addresses to features: %.2f seconds\n' % (time.time() - t))
    
    # Remove features that are None are contain nan values.

    for name in features_by_label.keys():
        valid = [(ftr, addr) for ftr, addr in zip(features_by_label[name], addresses_by_label[name])
                    if ftr is not None and not np.any(np.isnan(ftr))]
        res = zip(*valid)
        features_by_label[name] = np.array(res[0])
        addresses_by_label[name] = res[1]
    
    return features_by_label, addresses_by_label


In [15]:

# Generate training data set
features, addresses = generate_dataset_cell_based(num_samples_per_label=num_samples_per_label, 
                                                  stacks=stacks,
                                                  labels_to_sample=labels_to_sample,
                                                 cell_features_used=cell_features_used)
# Save training data set to '/shared/CSHL_cells_v2/classifiers/datasets/'
features_fp = os.path.join(CELL_FEATURES_CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_features.hdf')
create_parent_dir_if_not_exists(features_fp)
save_hdf_v2(features, features_fp)
upload_to_s3(features_fp)
# train_feat_dir = create_if_not_exists(os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset, 'patch_features'))
# for label, feats in training_features.iteritems():
#     bp.pack_ndarray_file(feats, os.path.join(train_feat_dir, label + '.bp'))

    # Save addresses
addresses_fp = os.path.join(CELL_FEATURES_CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_addresses.pkl')
save_pickle(addresses, addresses_fp)
upload_to_s3(addresses_fp)

'No object named structures in the file'


Annotation has no structures.
Load annotation. Time: 1.32 seconds.
File /shared/CSHL_cells_v2/classifiers/region_indices_by_label/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_region_indices_by_label.hdf does not exist
Cannot load pre-computed region labels for section 92... re-compute


NameError: global name 'get_margin_from_surround_label' is not defined

In [8]:
len(features['10N'])

10

In [16]:
addresses['10N']

NameError: name 'addresses' is not defined

In [17]:
for l, v in sorted(addresses.items()):
    print l, len(v)

NameError: name 'addresses' is not defined